1. Read json file
2. Set up the sampler through batchrunner
3. generate the input file
4. save it.

In [14]:
# Import relevant libraries
import os
from ModularCirc import BatchRunner
import numpy as np
import pandas as pd
from ModularCirc.Models.NaghaviModel import NaghaviModel, NaghaviModelParameters, TEMPLATE_TIME_SETUP_DICT
from SALib.analyze.sobol import analyze
from SALib.sample.sobol import sample as sobol_sample
from SALib.analyze.sobol import analyze as sobol_analyze
import matplotlib.pyplot as plt
from autoemulate.compare import AutoEmulate
from SALib.analyze.sobol import analyze
import numpy as np
import json

In [15]:
os.getcwd()

'/Users/syonis/project/comparative-gsa/sandbox'

In [16]:
# Define variables
param_path = "./parameters_naghavi.json"

n_samples = 4096 # Powers of 2 (n+2)1024 32768

simulation_out_path = f'outputs/simulations/output_{n_samples}_samples_fixed_T_constrained_no_overlap_venc/'

os.makedirs(os.path.join(simulation_out_path, 
                         f'raw_simulations_{n_samples}_samples'), 
                         exist_ok=True)
## read in save parameters to the folder where simulations r saved.
with open(param_path, 'r') as f: 
    params = json.load(f)
# save parameters to the folder where simulations are saved.
with open(os.path.join(simulation_out_path, 'parameters.json'), 'w') as f:
    json.dump(params, f, indent=4)

In [17]:
# Set up the batch runner
br = BatchRunner('Sobol', 0) # why are we using 'Sobol' here? 
br.setup_sampler(param_path)
br.sample(n_samples)

In [18]:
br.samples

,ao.r,ao.c,art.r,art.c,ven.r,ven.c,av.r,mv.r,la.E_pas,la.E_act,...,la.v,la.delay,la.t_tr,la.tau,la.t_max,lv.delay,lv.t_tr,lv.tau,lv.t_max,T
0,324.140512,0.429410,970.557289,2.593650,5.759509,136.536660,5.535208,3.645771,0.306149,0.687739,...,93,150,225,25,150,0,420,25,280,800
1,184.535318,0.260801,1633.145015,3.563218,9.590858,107.807678,8.180606,4.796574,0.377667,0.551798,...,93,150,225,25,150,0,420,25,280,800
2,166.084035,0.345046,672.740771,1.861618,13.262328,120.550115,3.269592,5.126834,0.388891,0.531352,...,93,150,225,25,150,0,420,25,280,800
3,281.490014,0.176454,1364.216930,4.353844,7.106007,148.472007,7.374171,2.190163,0.329412,0.637287,...,93,150,225,25,150,0,420,25,280,800
4,268.354010,0.301167,1520.599799,3.951997,8.580898,142.570080,4.480216,5.736952,0.403544,0.580024,...,93,150,225,25,150,0,420,25,280,800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4091,268.256424,0.241766,1160.750970,2.576748,9.169148,143.355731,7.616555,4.817270,0.401045,0.560122,...,93,150,225,25,150,0,420,25,280,800
4092,281.590146,0.403284,1565.699530,2.922108,11.073650,150.820415,4.535101,3.110839,0.336270,0.622509,...,93,150,225,25,150,0,420,25,280,800
4093,166.133084,0.272754,892.948375,3.234692,4.848106,126.231023,8.633637,4.274287,0.417615,0.488441,...,93,150,225,25,150,0,420,25,280,800
4094,184.493787,0.319029,1272.326862,2.189343,7.878021,111.922942,3.910122,5.661704,0.357745,0.588114,...,93,150,225,25,150,0,420,25,280,800


In [19]:
map_ = {
    'lv.t_tr' : ['lv.t_tr',],
    'la.t_tr' : ['la.t_tr',],
    'la.delay' : ['la.delay',],
    'lv.tau' : ['lv.tau',],
    'la.tau' : ['la.tau',],
    'lv.t_max' : ['lv.t_max',],
    'la.t_max' : ['la.t_max',],
}
br.map_sample_timings(
    ref_time=1000., # double check if 1000 or 1
    map=map_
    )

In [20]:
br.map_vessel_volume()

In [21]:
br.samples

,ao.r,ao.c,art.r,art.c,ven.r,ven.c,av.r,mv.r,la.E_pas,la.E_act,...,la.tau,la.t_max,lv.delay,lv.t_tr,lv.tau,lv.t_max,T,ao.v,art.v,ven.v
0,324.140512,0.429410,970.557289,2.593650,5.759509,136.536660,5.535208,3.645771,0.306149,0.687739,...,20.0,120.0,0,336.0,20.0,224.0,800,12.399427,74.892982,3942.565758
1,184.535318,0.260801,1633.145015,3.563218,9.590858,107.807678,8.180606,4.796574,0.377667,0.551798,...,20.0,120.0,0,336.0,20.0,224.0,800,11.886581,162.401792,4913.581178
2,166.084035,0.345046,672.740771,1.861618,13.262328,120.550115,3.269592,5.126834,0.388891,0.531352,...,20.0,120.0,0,336.0,20.0,224.0,800,16.974209,91.580607,5930.354665
3,281.490014,0.176454,1364.216930,4.353844,7.106007,148.472007,7.374171,2.190163,0.329412,0.637287,...,20.0,120.0,0,336.0,20.0,224.0,800,5.589738,137.922074,4703.330015
4,268.354010,0.301167,1520.599799,3.951997,8.580898,142.570080,4.480216,5.736952,0.403544,0.580024,...,20.0,120.0,0,336.0,20.0,224.0,800,9.354262,122.749416,4428.240972
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4091,268.256424,0.241766,1160.750970,2.576748,9.169148,143.355731,7.616555,4.817270,0.401045,0.560122,...,20.0,120.0,0,336.0,20.0,224.0,800,7.398941,78.858197,4387.225830
4092,281.590146,0.403284,1565.699530,2.922108,11.073650,150.820415,4.535101,3.110839,0.336270,0.622509,...,20.0,120.0,0,336.0,20.0,224.0,800,12.944200,93.790883,4840.882292
4093,166.133084,0.272754,892.948375,3.234692,4.848106,126.231023,8.633637,4.274287,0.417615,0.488441,...,20.0,120.0,0,336.0,20.0,224.0,800,12.527170,148.564341,5797.593044
4094,184.493787,0.319029,1272.326862,2.189343,7.878021,111.922942,3.910122,5.661704,0.357745,0.588114,...,20.0,120.0,0,336.0,20.0,224.0,800,14.399971,98.820098,5051.852445


In [22]:
# Save the samples to a CSV file

br.samples.to_csv(os.path.join(simulation_out_path,
                               f'input_samples_{n_samples}.csv'),
                                 index=False)


In [23]:
br.setup_model(model=NaghaviModel, po=NaghaviModelParameters,
                   time_setup=TEMPLATE_TIME_SETUP_DICT)

In [24]:
simulations = br.run_batch(
            n_jobs=8,
            output_path=os.path.join(simulation_out_path, f'raw_simulations_{n_samples}_samples') 
        )

100%|██████████| 4096/4096 [05:23<00:00, 12.64it/s]


In [25]:
import json

bool_indices = [index for index, value in enumerate(simulations) if isinstance(value, bool)]

with open(os.path.join(simulation_out_path,"bool_indices.json"), 'w') as f:
    json.dump(bool_indices, f)

print(bool_indices)  

[3910]


In [26]:
## Setting up the headers of the columns for my simulation summary similar to fay
##headers = [
   # 'v_ao', 'v_art', 'v_ven', 'v_la', 'v_lv', 'p_lv', 'q_av', 'p_ao', 'p_art',
   # 'q_ao', 'p_ven', 'q_art', 'p_la', 'q_ven', 'q_mv', 'T'
#]

summary_rows = []

for i_sim in range(len(simulations)): ## stuck with the length of simulations like fay
    sim_df = simulations[i_sim]
    
    # Skip if not a DataFrame this was needed because sometimes simulations can return None or other types. or false.
    if not isinstance(sim_df, pd.DataFrame):
        print(f"Simulation {i_sim} is not a DataFrame, skipping.")
        continue

    row = {} ##the 'dictiionary' to hold the summary for each simulation
    for h in sim_df.columns: # We had sim_df the whole time which means h can be used without the headers list!
        row[f"{h}_mean"] = sim_df[h].mean()
        row[f"{h}_max"] = sim_df[h].max() ## using the headers defined above to make the new csv file headers eg v_ao_mean, v_ao_max, etc.
    summary_rows.append(row) # this is a list that is being used to collect summary data for our simulations.
    print(f"Simulation {i_sim} done.") # simple way to track progress 

summary_df = pd.DataFrame(summary_rows)
summary_df.to_csv(os.path.join(simulation_out_path, "simulations_summary.csv"), index=False)  ## the location, nice and organised!
print("Saved as simulation_summary.csv") # not really needed but nice to show the file has been saved! :D

Simulation 0 done.
Simulation 1 done.
Simulation 2 done.
Simulation 3 done.
Simulation 4 done.
Simulation 5 done.
Simulation 6 done.
Simulation 7 done.
Simulation 8 done.
Simulation 9 done.
Simulation 10 done.
Simulation 11 done.
Simulation 12 done.
Simulation 13 done.
Simulation 14 done.
Simulation 15 done.
Simulation 16 done.
Simulation 17 done.
Simulation 18 done.
Simulation 19 done.
Simulation 20 done.
Simulation 21 done.
Simulation 22 done.
Simulation 23 done.
Simulation 24 done.
Simulation 25 done.
Simulation 26 done.
Simulation 27 done.
Simulation 28 done.
Simulation 29 done.
Simulation 30 done.
Simulation 31 done.
Simulation 32 done.
Simulation 33 done.
Simulation 34 done.
Simulation 35 done.
Simulation 36 done.
Simulation 37 done.
Simulation 38 done.
Simulation 39 done.
Simulation 40 done.
Simulation 41 done.
Simulation 42 done.
Simulation 43 done.
Simulation 44 done.
Simulation 45 done.
Simulation 46 done.
Simulation 47 done.
Simulation 48 done.
Simulation 49 done.
Simulation